In [1]:
import pandas as pd
import re
import numpy as np
from src import functions
import datetime

In [2]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x000001FB4CAC1598; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


### Tabla Detalle

In [13]:
prueba = functions.detalle(cur)

In [15]:
prueba.to_excel('./output/detalle_cartera_20200127.xlsx',encoding='CP1252',index=False)

# Base

In [3]:
df = functions.base(cur)

In [4]:
print('Filas Cartera:', len(df))
print('Total ventas:', len(df[df['isSold']==1]))
print('Total cancelaciones:', len(df[df['isCancelled']==1]))
print('Total assets:', len(df[df['isAsset']==1]))
print('Total bajas:', len(df[df['isBaja']==1]))
print('Total assets activos:', len(df[df['isActive']==1]))

Filas Cartera: 1274
Total ventas: 1274
Total cancelaciones: 44
Total assets: 1142
Total bajas: 218
Total assets activos: 924


In [6]:
df.to_excel('./output/df.xlsx',encoding='CP1252',index=False)

# VENTAS

In [4]:
functions.ass_vta_mes(df,tipo=5)

2018-10  2018-11  2018-12  \
canal_venta   Family       product_name                                         
DHO           X FIBER      X Fiber 100 - neba             0        0        0   
                           X Fiber 100 - own              0        0        1   
                           X Fiber 200 - neba             0        0        0   
                           X Fiber 200 - own              0        0        0   
                           X Fiber 50 - neba              0        1        1   
                           X Fiber 50 - own               0        2        1   
              X MUSIC      X Music                        2        1        3   
              X PRIVACY    X Privacy 100                  0        0        0   
                           X Privacy 200                  0        0        0   
                           X Privacy 50                   0        5        0   
              X PROTECTION X Protection                   0        0        1   
              X SECURITY   X Security 100                 0        0        0   
                           X Security 200                 0        0        0   
              X UCOM       Teamwork+PBX                   0        5        0   
IT integrator X FIBER      X Fiber 100 - neba             0        0        0   
                           X Fiber 100 - own              0        0        3   
                           X Fiber 200 - neba             0        1        0   
                           X Fiber 200 - own              0        0        1   
                           X Fiber 50 - neba              0        0        0   
                           X Fiber 50 - own               0        0        0   
              X MUSIC      X Music                        0        0        0   
              X PRIVACY    X Privacy 200                  0        0        1   
                           X Privacy 50                   0        0        1   
              X PROTECTION X Protection                   0        0        0   
              X SECURITY   X Security 100                 0        0        2   
                           X Security 200                 0        0        0   
              X UCOM       Teamwork+PBX                   0        0        6   
WEB           X FIBER      X Fiber 100 - own              0        0        0   
                           X Fiber 200 - neba             0        0        1   
                           X Fiber 200 - own              0        0        0   
                           X Fiber 50 - own               0        0        0   
              X MUSIC      X Music                        1        1        0   
              X PRIVACY    X Privacy 100                  0        0        0   
                           X Privacy 50                   0        0        2   
              X PROTECTION X Protection                   0        0        3   
              X SECURITY   X Security Monosede 200        0        0        0   
              X UCOM       Teamwork+PBX                   0        8        7   

                                                    2019-01  2019-02  2019-03  \
canal_venta   Family       product_name                                         
DHO           X FIBER      X Fiber 100 - neba             1        2        0   
                           X Fiber 100 - own              1        2        0   
                           X Fiber 200 - neba             0        1        0   
                           X Fiber 200 - own              0        0        0   
                           X Fiber 50 - neba              0        3        0   
                           X Fiber 50 - own               2        0        1   
              X MUSIC      X Music                        3        7        1   
              X PRIVACY    X Privacy 100                  0        2        0   
                           X Privacy 200                  0        0        0   
 

In [5]:
functions.cli_vta_mes(df,1)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
Family,,,,,,,,,,,,,,,,
X FIBER,0,3,8,7,7,2,8,6,9,16,4,8,1,2,3,3
X MUSIC,3,2,3,2,4,2,0,6,0,0,0,3,0,0,0,1
X PRIVACY,0,2,2,0,5,7,2,2,5,3,1,3,2,0,1,3
X PROTECTION,0,0,4,2,5,5,8,4,15,17,5,12,17,12,4,3
X SECURITY,0,0,2,2,0,0,0,2,5,3,0,4,3,1,1,1
X UCOM,0,4,4,5,6,7,5,11,19,17,6,22,27,28,29,25


# Altas

In [4]:
df_altas = functions.base_altas(df)

In [8]:
functions.cli_altas_mes_agg(df_altas,0)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
isClient,3,5,18,13,20,14,18,21,36,36,21,40,44,39,25,31


In [5]:
functions.cli_altas_mes(df,3)

2018-10  2018-11  2018-12  2019-01  2019-02  \
canal_venta   Family                                                      
DHO           X FIBER             0        0        3        4        4   
              X MUSIC             2        1        3        1        4   
              X PRIVACY           0        1        1        0        0   
              X PROTECTION        0        0        1        2        5   
              X SECURITY          0        0        0        0        0   
              X UCOM              0        0        1        2        7   
IT integrator X FIBER             0        1        3        2        0   
              X MUSIC             0        0        0        1        0   
              X PRIVACY           0        0        0        0        1   
              X PROTECTION        0        0        0        0        0   
              X SECURITY          0        0        0        2        1   
              X UCOM              0        0        1        0        0   
WEB           X FIBER             0        0        0        1        1   
              X MUSIC             1        1        0        0        0   
              X PRIVACY           0        0        0        1        0   
              X PROTECTION        0        0        3        0        0   
              X SECURITY          0        0        0        0        0   
              X UCOM              0        1        3        1        1   

                            2019-03  2019-04  2019-05  2019-06  2019-07  \
canal_venta   Family                                                      
DHO           X FIBER             3        2        5        8        5   
              X MUSIC             1        0        0        0        0   
              X PRIVACY           4        6        0        6        1   
              X PROTECTION        4        2        5       14       11   
              X SECURITY          0        0        0        1        3   
              X UCOM              2        2        6       10       17   
IT integrator X FIBER             1        4        1        4        4   
              X MUSIC             0        0        1        0        0   
              X PRIVACY           0        0        0        0        0   
              X PROTECTION        0        3        0        0        0   
              X SECURITY          0        0        1        1        3   
              X UCOM              1        2        1        0        0   
WEB           X FIBER             1        0        0        1        1   
              X MUSIC             1        0        5        0        0   
              X PRIVACY           0        1        0        0        0   
              X PROTECTION        1        2        0        1        0   
              X SECURITY          0        0        0        0        0   
              X UCOM              4        2        1        3        4   

                            2019-08  2019-09  2019-10  2019-11  2019-12  \
canal_venta   Family                                                      
DHO           X FIBER             4        6        1        1        0   
              X MUSIC             0        1        0        0        0   
              X PRIVACY           3        3        3        0        0   
              X PROTECTION       11        9       16       10        4   
              X SECURITY          1        0        4        1        1   
              X UCOM              5       16       18       26       27   
IT integrator X FIBER             2        3        1        2        1   
              X MUSIC             0        1        0        0        0   
              X PRIVACY           0        0        0        0        0   
              X PROTECTION        0        2        0        1        0   
              X SECURITY          0        1        1        2        1   
              X UCOM              0        0       

In [18]:
functions.ass_altas_mes(df,5)

2018-10  2018-11  2018-12  \
canal_venta   Family       product_name                                         
DHO           X FIBER      X Fiber 100 - neba             0        0        0   
                           X Fiber 100 - own              0        0        0   
                           X Fiber 200 - neba             0        0        0   
                           X Fiber 200 - own              0        0        0   
                           X Fiber 50 - neba              0        0        2   
                           X Fiber 50 - own               0        0        2   
              X MUSIC      Assets                         0        0        0   
                           X Music 4Gb                    2        1        3   
              X PRIVACY    X Privacy 100                  0        0        0   
                           X Privacy 200                  0        0        0   
                           X Privacy 50                   0        3        2   
              X PROTECTION Assets                         0        0        1   
              X SECURITY   X Security 100                 0        0        0   
                           X Security 200                 0        0        0   
              X UCOM       Teamwork+PBX                   0        0        5   
IT integrator X FIBER      X Fiber 100 - neba             0        0        0   
                           X Fiber 100 - own              0        0        3   
                           X Fiber 200 - neba             0        1        0   
                           X Fiber 200 - own              0        0        0   
                           X Fiber 50 - neba              0        0        0   
                           X Fiber 50 - own               0        0        0   
              X MUSIC      Assets                         0        0        0   
                           X Music 4Gb                    0        0        0   
              X PRIVACY    X Privacy 200                  0        0        0   
                           X Privacy 50                   0        0        0   
              X PROTECTION Assets                         0        0        0   
              X SECURITY   X Security 100                 0        0        0   
                           X Security 200                 0        0        0   
              X UCOM       Teamwork+PBX                   0        0        6   
WEB           X FIBER      X Fiber 100 - own              0        0        0   
                           X Fiber 200 - neba             0        0        0   
                           X Fiber 200 - own              0        0        0   
                           X Fiber 50 - own               0        0        0   
              X MUSIC      Assets                         0        0        0   
                           X Music 4Gb                    1        1        0   
              X PRIVACY    X Privacy 100                  0        0        0   
                           X Privacy 50                   0        0        0   
              X PROTECTION Assets                         0        0        3   
              X SECURITY   X Security Monosede 200        0        0        0   
              X UCOM       Teamwork+PBX                   0        5        5   

                                                    2019-01  2019-02  2019-03  \
canal_venta   Family       product_name                                         
DHO           X FIBER      X Fiber 100 - neba             1        1        0   
                           X Fiber 100 - own              2        1        0   
                           X Fiber 200 - neba             0        0        0   
                           X Fiber 200 - own              0        0        0   
                           X Fiber 50 - neba              0        1        2   
                           X Fiber 50 - own               1        1        1   
 

In [4]:
functions.cli_bajas_mes(df,3).reset_index()

,canal_venta,Family,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
0,DHO,X FIBER,0,0,0,0,0,1,0,0,0,2,1,1,1,0,1,1
1,DHO,X MUSIC,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
2,DHO,X PRIVACY,0,0,0,0,0,0,2,3,1,1,2,1,3,0,0,0
3,DHO,X PROTECTION,0,0,0,0,0,0,0,2,3,0,1,1,1,2,2,4
4,DHO,X SECURITY,0,0,0,0,0,0,0,0,0,1,0,0,2,0,2,0
5,DHO,X UCOM,0,0,0,1,1,2,1,1,2,4,3,2,4,3,5,6
6,IT integrator,X FIBER,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0
7,IT integrator,X MUSIC,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
8,IT integrator,X PROTECTION,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0
9,IT integrator,X SECURITY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [46]:
df_altas.head()

,cif,canal_venta,assets_act,rfb,deactivation_date,assets_tot,isClient,isActive,assets_dea,isBaja
0,03817373V,WEB,0,2019-11-20 00:00:01,2019-11-20 15:29:08,2,1,0,2,1
1,07477239P,DHO,0,2018-10-25 09:38:28,2019-01-21 09:13:32,1,1,0,1,1
2,11058321J,DHO,1,2019-01-23 00:00:00,NaT,1,1,1,0,0
3,12371541W,WEB,0,2018-12-18 00:00:01,2019-01-02 12:15:09,1,1,0,1,1
4,16601899Q,DHO,1,2019-05-22 00:00:01,NaT,5,1,1,4,0


In [96]:
functions.ass_bajas_mes(df, tipo=1)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
Family,,,,,,,,,,,,,,,,
X FIBER,0,0,0,0,0,2,0,0,1,3,3,3,2,0,2,2
X MUSIC,0,0,1,1,0,1,2,2,0,0,1,1,0,0,0,0
X PRIVACY,0,0,0,0,0,0,6,4,2,2,10,2,5,0,0,0
X PROTECTION,0,0,0,2,0,0,2,3,4,0,1,2,1,13,2,6
X SECURITY,0,0,0,0,0,0,0,0,0,2,0,0,3,0,3,1
X UCOM,0,0,0,3,2,6,1,3,7,11,9,19,14,7,13,20


## Migras

In [5]:
mig_df=functions.migBase(cur)
mig_df.head(2)

,assetid,assethijo_mig,product_name,Family,cif,canal_venta,mig_date,tipo_migra
0,02i0N00000I3wptQAB,None,Teamwork+PBX,X UCOM,B98115520,DHO,2019-05-09 17:21:03,IN
1,02i0N00000KmJaoQAF,None,Teamwork+PBX,X UCOM,B78909611,DHO,2019-07-19 16:34:33,IN


In [6]:
migin, migout = functions.mig_producto_mes(mig_df)

In [7]:
migin

2018-10  2018-11  2018-12  2019-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              0        0        0        0   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2019-02  2019-03  2019-04  2019-05  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        1        0   
              X UCOM  Teamwork+PBX              0        0        0       11   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2019-06  2019-07  2019-08  2019-09  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              0        2        1        1   
IT integrator X UCOM  Teamwork+PBX              1        0        0        0   

                                          2019-10  2019-11  2019-12  2020-01  
canal_venta   Family  product_name                                            
DHO           X FIBER X Fiber 100 - neba        0        0        0        0  
              X UCOM  Teamwork+PBX              3        1        0        1  
IT integrator X UCOM  Teamwork+PBX              0        0        0        0

In [8]:
migout

2018-10  2018-11  2018-12  2019-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 200 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              0        0        0        0   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2019-02  2019-03  2019-04  2019-05  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 200 - neba        0        0        1        0   
              X UCOM  Teamwork+PBX              0        0        0       11   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2019-06  2019-07  2019-08  2019-09  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 200 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              0        2        1        1   
IT integrator X UCOM  Teamwork+PBX              1        0        0        0   

                                          2019-10  2019-11  2019-12  2020-01  
canal_venta   Family  product_name                                            
DHO           X FIBER X Fiber 200 - neba        0        0        0        0  
              X UCOM  Teamwork+PBX              3        1        0        1  
IT integrator X UCOM  Teamwork+PBX              0        0        0        0

In [33]:
getVelocityFrom('20')

'Portable 20'